# Gather Information from Michigan Parent Dashboard

The goal of this script is to create a single data frame containing all of the school offering links in the Michigan Parent Dashboard. Once run, the dataframe will have the following columns: district_code, the school name, the school's location id [digital], and school offering/services link.


DOI plans. Once run, the dataframe will have four columns: district, file type, level, link.

# Set-Up

In [1]:
import gather_documents
import bs4 as bs
import urllib.request
import os
import pandas as pd
import random
from urllib.parse import urljoin, quote
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC    
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
test_sample = False
sample_size = 15

In [3]:
data_home = "C:/Users/johnw/Box/Michigan Parent Dashboard/Website Scraping"

In [4]:
#Display Settings
pd.set_option('display.max_colwidth', None)

# Clean District_ID

In [5]:
unformatted_district_id = "01010 02010 02020 02070 02080 03000 03010 03020 03030 03040 03050 03060 03070 03080 03100 03440 03900 03902 04000 04010 05010 05035 05040 05060 05065 05070 06020 06050 07010 07020 07040 08000 08010 08030 08050 09000 09010 09030 09050 09090 09901 09902 09903 10015 10025 11000 11010 11020 11030 11033 11200 11210 11240 11250 11300 11310 11320 11330 11340 11670 11830 11900 11901 11903 11904 12000 12010 12020 12040 12901 13000 13020 13050 13070 13080 13090 13095 13110 13120 13130 13135 13900 13901 13902 13903 13904 14000 14010 14020 14030 14050 15000 15010 15020 15030 15050 15060 15901 15902 16000 16015 16050 16070 16100 17000 17010 17050 17090 17110 17140 17160 17900 17901 17902 17903 18000 18010 18020 18060 19000 19010 19070 19100 19120 19125 19140 19900 20015 21000 21010 21025 21060 21065 21090 21135 22000 22010 22025 22030 22045 23000 23010 23030 23050 23060 23065 23080 23090 23490 23900 23901 23902 23903 24020 24030 24040 24070 24901 25000 25010 25030 25040 25050 25060 25070 25080 25100 25110 25120 25130 25140 25150 25180 25200 25210 25230 25240 25250 25260 25280 25902 25903 25904 25905 25907 25909 25910 25911 25912 25914 25915 25916 25918 25919 26010 26040 27000 27010 27020 27070 27080 28000 28010 28035 28090 28900 28901 28902 28904 29000 29010 29020 29040 29050 29060 29100 30000 30010 30020 30030 30040 30050 30060 30070 30080 30901 30902 31000 31010 31020 31030 31050 31070 31100 31110 31130 31140 31900 32000 32010 32030 32040 32050 32060 32080 32090 32170 32260 32610 32620 32650 33000 33010 33020 33040 33060 33070 33100 33130 33170 33200 33215 33220 33230 33901 33904 33906 33909 33910 33911 33914 33915 34000 34010 34080 34090 34110 34120 34140 34340 34360 35000 35010 35020 35030 35040 35902 36015 36025 37010 37040 37060 37900 37901 38000 38010 38020 38040 38050 38080 38090 38100 38120 38130 38140 38150 38170 38900 38901 38902 38904 39000 39010 39020 39030 39050 39065 39130 39140 39160 39170 39903 39905 39906 39907 39911 40020 40040 40060 41000 41010 41020 41025 41026 41040 41050 41070 41080 41090 41110 41120 41130 41140 41145 41150 41160 41170 41210 41240 41900 41901 41904 41905 41908 41909 41910 41911 41914 41915 41916 41917 41918 41919 41920 41921 41922 41923 41924 41925 41926 41928 41929 41930 41931 42030 43040 44000 44010 44020 44050 44060 44090 44901 45010 45020 45040 45050 45901 46000 46010 46020 46040 46050 46060 46080 46090 46100 46110 46130 46140 47000 47010 47030 47060 47070 47080 47900 47901 47902 47903 48040 49010 49020 49040 49055 49070 49110 49901 50000 50010 50020 50030 50040 50050 50070 50080 50090 50100 50120 50130 50140 50160 50170 50180 50190 50200 50210 50220 50230 50240 50902 50903 50905 50906 50908 50909 50911 50912 50913 50914 50918 50920 51000 51020 51045 51060 51070 51903 51905 52000 52015 52040 52090 52100 52110 52160 52170 52180 52901 53000 53010 53020 53040 53901 54000 54010 54025 54040 54901 55000 55010 55100 55115 55120 55900 55901 56000 56010 56020 56030 56050 56901 56903 57020 57030 58000 58010 58020 58030 58050 58070 58080 58090 58100 58110 58901 58902 59000 59020 59045 59070 59080 59090 59125 59150 59900 60010 60020 61000 61010 61060 61065 61080 61120 61180 61190 61210 61220 61230 61240 61900 61902 61904 61905 61906 61907 62000 62040 62050 62060 62070 62090 62470 63000 63010 63020 63030 63040 63050 63060 63070 63080 63090 63100 63110 63130 63140 63150 63160 63180 63190 63200 63210 63220 63230 63240 63250 63260 63270 63280 63290 63300 63900 63901 63906 63907 63909 63910 63911 63912 63913 63914 63915 63917 63918 63920 63921 63922 63923 63924 63926 63928 63929 63931 63934 63938 63939 64040 64070 64080 64090 65045 65900 66045 66050 67020 67050 67055 67060 68010 68030 69020 69030 69040 70000 70010 70020 70040 70070 70120 70175 70190 70300 70350 70901 70902 70904 70905 70906 70908 70909 71050 71060 71080 71902 72000 72010 72020 72901 73000 73010 73030 73040 73110 73170 73180 73190 73200 73210 73230 73240 73255 73900 73901 73908 73909 73910 73912 74000 74010 74030 74040 74050 74100 74120 74130 74900 74903 74911 74912 74914 75000 75010 75020 75030 75040 75050 75060 75070 75080 75100 76000 76060 76070 76080 76090 76140 76180 76210 77010 78000 78020 78030 78040 78060 78070 78080 78100 78110 79000 79010 79020 79030 79080 79090 79100 79110 79145 79150 80000 80010 80020 80040 80050 80090 80110 80120 80130 80140 80150 80160 80240 80900 81000 81010 81020 81040 81050 81070 81080 81100 81120 81140 81900 81901 81902 81903 81905 81906 81908 81909 81910 81911 81912 81913 81915 82015 82020 82030 82040 82045 82050 82055 82060 82090 82095 82100 82110 82120 82130 82140 82150 82155 82160 82170 82180 82230 82240 82250 82290 82300 82320 82340 82365 82390 82400 82405 82430 82700 82701 82702 82703 82704 82705 82706 82710 82711 82713 82716 82717 82718 82719 82722 82723 82724 82725 82727 82728 82729 82730 82735 82737 82739 82742 82743 82744 82745 82746 82747 82748 82749 82751 82752 82754 82756 82757 82758 82760 82762 82763 82765 82766 82767 82770 82772 82773 82774 82775 82776 82904 82910 82915 82916 82918 82919 82921 82923 82924 82925 82926 82928 82929 82930 82933 82937 82938 82940 82941 82942 82943 82945 82947 82950 82953 82956 82957 82958 82959 82963 82967 82968 82969 82970 82973 82974 82975 82976 82977 82979 82981 82982 82983 82985 82986 82987 82991 82994 82995 82996 82997 83000 83010 83060 83070 83900 84010 84050"

In [6]:
# Remove leading and trailing spaces
unformatted_district_id = unformatted_district_id.strip()

# Split the district IDs by space
district_list = unformatted_district_id.split(" ")

# Iterate over the district IDs and group them into sets of five digits
formatted_district_list = [district[i:i+5] for district in district_list for i in range(0, len(district), 5)]

# Stage 1: Scrape All Schools in Michigan

The parent dashboard is unfortunately fickle. It does not 'properly' utilize each school's building code. Instead, there is some sort of generated school location id combination that is not easily identifiable for an individual school. However, it appears that the district code [id] can be used to navigate the various parent dashboards. As such, Stage 1 systematically iterates through all district id's and scrapes all schools within the district. In particular, I collect the district_id, district_name, core_link [starting link for the district], school_level [e.g., Elementary], school_name, school grade [e.g., K-5], and the school_link. The school_link is the most important for progressing to the next stage. It contains the parent dashboard for that individual school- while it is unlikely that I will scrape this main parent dashboard (information should be available from public datasets), the "School Services and Offerings" is of interest for the next stage.

In [7]:
# Desired Columns from Stage 1
columns = ['District ID', 'District Name', 'Core Link', 'School Level', 'School Name', 'School Grade', 'School Link']

# Create an empty DataFrame with column labels
Stage1 = pd.DataFrame(columns=columns)

In [8]:
random.seed(42)

# Extract a sample of 3 district codes
sample = random.sample(formatted_district_list, sample_size)

# Print the sample
print(sample)

['73908', '17903', '05070', '82130', '35000', '32650', '31010', '23030', '82090', '17000', '78030', '82120', '63160', '14030', '66050']


In [10]:
# Base URL
base_url = 'https://legacy.mischooldata.org'

# Loop through the district_ids
for district_id in formatted_district_list[673:]:

    # Perform some operation on each district code
    print("Processing district code:", district_id)
    
    # Setting Core Link:
    initial_link = 'https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=DISTRICT_CODE&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools'
    district_link = initial_link.replace("DISTRICT_CODE", district_id)
    
    source = urllib.request.urlopen(district_link).read()
    soup = bs.BeautifulSoup(source, 'html')

    # Find the div element with class "DistrictDisplay"
    district_div = soup.find('div', class_='DistrictDisplay')

    # Initialize district_name and flag_empty_district
    district_name = ""
    flag_empty_district = False

    if district_div is not None:
        district_name = district_div.text.strip()
    else:
        flag_empty_district = True

    # Find all <div> tags with class "SchoolItem"
    school_items = soup.find_all('div', class_='SchoolItem')
    
    # Loop through each <div> tag
    for school_item in school_items:
        # Extract school details as before

        # Create a dictionary with the row values
        row = {
            'District ID': district_id,
            'District Name': district_name,
            'Core Link': district_link,
            'School Level': school_level,
            'School Name': school_name,
            'School Grade': school_grade,
            'School Link': school_link
        }
        
        # Concatenating District ID, District Name, Core Link, School Level, School Name, School Grade, and School Link to Stage 1 Dataframe
        Stage1 = pd.concat([Stage1, pd.DataFrame.from_records([row])])

    if flag_empty_district:
        print("Warning: District name is empty for district code:", district_id)

Processing district code: 75020
Processing district code: 75030
Processing district code: 75040
Processing district code: 75050
Processing district code: 75060
Processing district code: 75070
Processing district code: 75080
Processing district code: 75100
Processing district code: 76000
Processing district code: 76060
Processing district code: 76070
Processing district code: 76080
Processing district code: 76090
Processing district code: 76140
Processing district code: 76180
Processing district code: 76210
Processing district code: 77010
Processing district code: 78000
Processing district code: 78020
Processing district code: 78030
Processing district code: 78040
Processing district code: 78060
Processing district code: 78070
Processing district code: 78080
Processing district code: 78100
Processing district code: 78110
Processing district code: 79000
Processing district code: 79010
Processing district code: 79020
Processing district code: 79030
Processing district code: 79080
Processi

In [11]:
Stage1

,District ID,District Name,Core Link,School Level,School Name,School Grade,School Link
0,75020,Burr Oak Community School District,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75020&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Elementary/Middle School,Burr Oak Elementary School,K-8,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Elem_Middle%20School&LocationId=S,6012,971,113&LocationCode=75020"
0,75020,Burr Oak Community School District,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75020&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Elementary through High School,Burr Oak Virtual Academy,K-12,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Elem%20thru%20High%20School&LocationId=S,16050,971,113&LocationCode=75020"
0,75020,Burr Oak Community School District,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75020&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",High School,Burr Oak High School,9-12,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&LocationId=S,6013,971,113&LocationCode=75020"
0,75030,Centreville Public Schools,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75030&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Elementary/Middle School,Centreville Elementary School,K-6,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Elem_Middle%20School&LocationId=S,6289,1002,113&LocationCode=75030"
0,75030,Centreville Public Schools,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75030&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Middle/High School,Centreville Jr. /Sr. High School,7-12,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Middle_High%20School&LocationId=S,6290,1002,113&LocationCode=75030"
...,...,...,...,...,...,...,...
0,83070,Mesick Consolidated Schools,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=83070&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Elementary/Middle School,East Arbor Charter Academy,K-8,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Elem_Middle%20School&LocationId=S,12631,1793,118&LocationCode=81910"
0,83900,Highpoint Virtual Academy of Michigan,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=83900&LocationType=District&

# Stage 2

It appears that the href for the School Services and Offerings is simply: /ParentDashboard/ParentDashboardSchoolServicesAndOfferings.aspx. As a result, this indicates to me that I will need to simply replace the "/ParentDashboard/ParentDashboardSchoolOverview.aspx" with the previous line.

In [12]:
#Updating Stage 2 to have existing data from Stage 1
Stage2 = Stage1.copy()

In [13]:
#Replacing the SchoolOverview line with SchoolServices
Stage2['Offering Link'] = Stage1['School Link'].str.replace('ParentDashboardSchoolOverview.aspx', 'ParentDashboardSchoolServicesAndOfferings.aspx')

C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3029717339.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  Stage2['Offering Link'] = Stage1['School Link'].str.replace('ParentDashboardSchoolOverview.aspx', 'ParentDashboardSchoolServicesAndOfferings.aspx')


In [14]:
Stage2

,District ID,District Name,Core Link,School Level,School Name,School Grade,School Link,Offering Link
0,75020,Burr Oak Community School District,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75020&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Elementary/Middle School,Burr Oak Elementary School,K-8,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Elem_Middle%20School&LocationId=S,6012,971,113&LocationCode=75020","https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolServicesAndOfferings.aspx?SchoolLevel=Elem_Middle%20School&LocationId=S,6012,971,113&LocationCode=75020"
0,75020,Burr Oak Community School District,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75020&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Elementary through High School,Burr Oak Virtual Academy,K-12,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Elem%20thru%20High%20School&LocationId=S,16050,971,113&LocationCode=75020","https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolServicesAndOfferings.aspx?SchoolLevel=Elem%20thru%20High%20School&LocationId=S,16050,971,113&LocationCode=75020"
0,75020,Burr Oak Community School District,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75020&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",High School,Burr Oak High School,9-12,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&LocationId=S,6013,971,113&LocationCode=75020","https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolServicesAndOfferings.aspx?SchoolLevel=High%20School&LocationId=S,6013,971,113&LocationCode=75020"
0,75030,Centreville Public Schools,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75030&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Elementary/Middle School,Centreville Elementary School,K-6,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Elem_Middle%20School&LocationId=S,6289,1002,113&LocationCode=75030","https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolServicesAndOfferings.aspx?SchoolLevel=Elem_Middle%20School&LocationId=S,6289,1002,113&LocationCode=75030"
0,75030,Centreville Public Schools,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchools.aspx?LocationCode=75030&LocationType=District&SchoolLevel=None&ReturnUrl=/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=High%20School&amp;LocationId=S,10298,1479,63&amp;LocationCode=03065:&ReturnName=Plainwell%20High%20School&&LocationName=Plainwell%20Community%20Schools",Middle/High School,Centreville Jr. /Sr. High School,7-12,"https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolOverview.aspx?SchoolLevel=Middle_High%20School&LocationId=S,6290,1002,113&LocationCode=75030","https://legacy.mischooldata.org/ParentDashboard/ParentDashboardSchoolServicesAndOfferings.aspx?Sch

In [15]:
Stage2.to_csv(f"{data_home}/Stage2_10_30_23.csv", index=True)

# Stage 3

In Stage 3, I will conduct an overarching gathering phase of the various elements on the School Services and Offering Page. In particular, it is important to acknowledge that the different school offerings, although a similar overarching structure, may or may not have elements displayed. For example, an Elementary school should not offer Advanced Placement courses; we may see an overarching header of Advanced Placement but there would be nothing underneath. High schools, which may offer Advanced Placement courses, simply display the ones that they offer (e.g., Calculus AB). Any AP courses that are not offered will not be listed.

If we're conceptualizing an eventual dataset with a binary indicator for each offering/service, I would need to generally know a priori all of the potential offerings. Except, it appears that AP courses have a strange, almost hand input quality. 

Adrian High School: AP AM HIST C; AP AM HIST A; AP AM HIST B
Novi High School: AP CHEMISTRY (A); MIVHS: AP STATISTICS (A); MIVHS: AP STATISTICS (B); MIVHS: AP U.S. GOVERNMENT (A); MIVHS: AP WORLD HISTORY: MODERN (A)
Troy High School: STATISTICS AP; PSYCHOLOGY AP; CHEMISTRY 2 AP

I think there is some obvious overlap, but the formatting will be completely off. I don't think it makes sense to try to clean this before I know the full extent of the possible variations that may occur. I know there is a finite number of AP courses, so I could pursue various routes to address this problem. 1) I could try to simply list out all possible variations and see if the text is contained (e.g., AP STATS, AP Statistics, Statistics AP)-- I don't think I could ever truly capture the full variation. 2) I could simply collect everything and have an ever-expanding dataset and then simply manually clean this afterwards. I think this is the feasible approach. I can easily identify that AP Statistics and Statistics AP are the same thing and then subsequently move that data into the same column.

It seems to me that CTE and IB are less of a concern since they follow the same overarching structure of Arts/Languages, etc. 

In [16]:
#Creation of Master School Services and Offering List
masterservicesofferinglist = []

In [17]:
#Updating Stage 3 to have existing data from Stage 2
Stage3 = Stage2.copy()
Stage3 = Stage3.reset_index()
Stage3['retry'] = 0

In [19]:
for i in range(1, len(Stage3)):

    # Perform some operation on each school
    print("Processing District ID:", Stage3.iloc[i]['District ID'], "School Name:", Stage3.iloc[i]['School Name'])

    # Set up the Selenium driver
    service = Service('C:\Program Files\chromedriver.exe')  # Specify the path to your chromedriver executable
    options = Options()
    options.add_argument('--headless')  # Run the browser in headless mode (without GUI)
    driver = webdriver.Chrome(service=service, options=options)
    
    #Identifying the appropriate website
    driver.get(Stage3.iloc[i]['Offering Link'])

    # Wait for the parent element to be present
    driver.implicitly_wait(5)

    # Find all the parent elements
    parent_elements = driver.find_elements(By.CSS_SELECTOR, '.SchoolServicesAndOfferingsGroupContent')

    # Iterate over each parent element
    for parent_element in parent_elements:
        try:
            # Find the description within the parent element
            description = parent_element.find_element(By.CSS_SELECTOR, '.SchoolServicesAndOfferingsDescription').get_attribute("innerText")

            # Find all the values within the parent element
            values = parent_element.find_elements(By.CSS_SELECTOR, 'ul.SchoolServicesAndOfferingsValues li')

            # Extract the texts and their corresponding status
            for value in values:
                try: 
                    text = value.get_attribute("innerText")
                    descservicesoffering = description + "-" + text

                    if descservicesoffering in masterservicesofferinglist:
                        if 'Disabled' in value.get_attribute('class'):
                            pass
                        else:
                            Stage3.loc[i,descservicesoffering] = 1
                    else:
                        masterservicesofferinglist.append(descservicesoffering)
                        Stage3[descservicesoffering] = 0

                        if 'Disabled' in value.get_attribute('class'):
                            pass
                        else:
                            Stage3.loc[i,descservicesoffering] = 1       

                except StaleElementReferenceException:

                    # Handle the exception if the element becomes stale
                    print("StaleElementReferenceException occurred. Future Retry Needed.")
                    Stage3.loc[i,'retry'] = 1
        except StaleElementReferenceException:

        # Handle the exception if the element becomes stale
            print("StaleElementReferenceException occurred. Future Retry Needed.")
            Stage3.loc[i,'retry'] = 1
            
    # Close the WebDriver
    driver.quit()   

Processing District ID: 75020 School Name: Burr Oak Virtual Academy
Processing District ID: 75020 School Name: Burr Oak High School
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
Processing District ID: 75030 School Name: Centreville Elementary School
Processing District ID: 75030 School Name: Centreville Jr. /Sr. High School
Processing District ID: 75030 School 

C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stage3[descservicesoffering] = 0
C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stage3[descservicesoffering] = 0
C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colu

Processing District ID: 75050 School Name: Constantine Alternative Education
Processing District ID: 75050 School Name: Constantine High School


C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stage3[descservicesoffering] = 0
C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stage3[descservicesoffering] = 0
C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colu

C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stage3[descservicesoffering] = 0
C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Stage3[descservicesoffering] = 0
C:\Users\johnw\AppData\Local\Temp\ipykernel_20348\3509051139.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colu

...

Processing District ID: 82752 School Name: East Arbor Charter Academy
Processing District ID: 82754 School Name: East Arbor Charter Academy
Processing District ID: 82754 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Futur

Processing District ID: 82916 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.


Processing District ID: 82943 School Name: East Arbor Charter Academy
Processing District ID: 82945 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry 

Processing District ID: 82970 School Name: East Arbor Charter Academy
Processing District ID: 82973 School Name: East Arbor Charter Academy
Processing District ID: 82974 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Futur

Processing District ID: 82996 School Name: East Arbor Charter Academy
Processing District ID: 82997 School Name: East Arbor Charter Academy
Processing District ID: 83000 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Futur

In [20]:
#Preparing for next iteration for the 647 schools whose data did not get scraped.
retry_rows = Stage3[Stage3['retry'] == 1].index.tolist()
Stage3['retry2'] = 0

In [21]:
for rt in retry_rows:
    
    # Perform some operation on each school
    print("Reprocessing District ID:", Stage3.iloc[rt]['District ID'], "School Name:", Stage3.iloc[rt]['School Name'])
    
    # Set up the Selenium driver
    service = Service('C:\Program Files\chromedriver.exe')  # Specify the path to your chromedriver executable
    options = Options()
    options.add_argument('--headless')  # Run the browser in headless mode (without GUI)
    driver = webdriver.Chrome(service=service, options=options)
    
    #Identifying the appropriate website
    driver.get(Stage3.iloc[rt]['Offering Link'])

    # Wait for the parent element to be present
    driver.implicitly_wait(2)

    # Find all the parent elements
    parent_elements = driver.find_elements(By.CSS_SELECTOR, '.SchoolServicesAndOfferingsGroupContent')

    # Iterate over each parent element
    for parent_element in parent_elements:
        try:
            # Find the description within the parent element
            description = parent_element.find_element(By.CSS_SELECTOR, '.SchoolServicesAndOfferingsDescription').get_attribute("innerText")

            # Find all the values within the parent element
            values = parent_element.find_elements(By.CSS_SELECTOR, 'ul.SchoolServicesAndOfferingsValues li')

            # Extract the texts and their corresponding status
            for value in values:
                try: 
                    text = value.get_attribute("innerText")
                    descservicesoffering = description + "-" + text

                    if descservicesoffering in masterservicesofferinglist:
                        if 'Disabled' in value.get_attribute('class'):
                            pass
                        else:
                            Stage3.loc[rt,descservicesoffering] = 1
                    else:
                        masterservicesofferinglist.append(descservicesoffering)
                        Stage3[descservicesoffering] = 0

                        if 'Disabled' in value.get_attribute('class'):
                            pass
                        else:
                            Stage3.loc[rt,descservicesoffering] = 1       

                except StaleElementReferenceException:

                    # Handle the exception if the element becomes stale
                    print("StaleElementReferenceException occurred. Future Retry Needed.")
                    Stage3.loc[rt,'retry2'] = 1
        except StaleElementReferenceException:

        # Handle the exception if the element becomes stale
            print("StaleElementReferenceException occurred. Future Retry Needed.")
            Stage3.loc[rt,'retry2'] = 1
            
    # Close the WebDriver
    driver.quit()  

Reprocessing District ID: 75020 School Name: Burr Oak High School
Reprocessing District ID: 75050 School Name: Riverside Elementary School
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
Reprocessing District ID: 75060 School Name: Mendon Middle/High School
Reprocessing District ID: 75070 School Name: C

Reprocessing District ID: 80000 School Name: Maple Creek Education Center
Reprocessing District ID: 80010 School Name: Maple Grove Elementary School
Reprocessing District ID: 80010 School Name: North Shore Elementary
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
Reprocessing District ID: 80010 School 

Reprocessing District ID: 81000 School Name: WISD Early On
Reprocessing District ID: 81010 School Name: Angell School
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
Reprocessing District ID: 81010 School Name: Bach Elementary School
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleEle

Reprocessing District ID: 81902 School Name: Central Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
Reprocessing District ID: 81906 School Name: Fortis Academy
Reprocessing District ID: 81909 School Name: Arbor Preparatory High School
Reprocessing District ID: 81910 School Name: East Arbor Char

Reprocessing District ID: 79000 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed

...

Reprocessing District ID: 82095 School Name: East Arbor Charter Academy
Reprocessing District ID: 82095 School Name: East Arbor Charter Academy
Reprocessing District ID: 82100 School Name: East Arbor Charter Academy
Reprocessing District ID: 82100 School Name: East Arbor Charter Academy
Reprocessing District ID: 82100 School Name: East Arbor Charter Academy
Reprocessing District ID: 82100 School Name: East Arbor Charter Academy
Reprocessing District ID: 82100 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.


Reprocessing District ID: 82160 School Name: East Arbor Charter Academy
Reprocessing District ID: 82160 School Name: East Arbor Charter Academy
Reprocessing District ID: 82160 School Name: East Arbor Charter Academy
Reprocessing District ID: 82160 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException

Reprocessing District ID: 82390 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed

Reprocessing District ID: 82930 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed

Reprocessing District ID: 83010 School Name: East Arbor Charter Academy
Reprocessing District ID: 83010 School Name: East Arbor Charter Academy
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Re

In [22]:
#Preparing for next iteration for the 96 schools whose data did not get scraped.
retry_rows2 = Stage3[Stage3['retry2'] == 1].index.tolist()
Stage3['retry3'] = 0

In [25]:
for rt in retry_rows2:
    
    # Perform some operation on each school
    print("Reprocessing District ID:", Stage3.iloc[rt]['District ID'], "School Name:", Stage3.iloc[rt]['School Name'])
    
    # Set up the Selenium driver
    service = Service('C:\Program Files\chromedriver.exe')  # Specify the path to your chromedriver executable
    options = Options()
    options.add_argument('--headless')  # Run the browser in headless mode (without GUI)
    driver = webdriver.Chrome(service=service, options=options)
    
    #Identifying the appropriate website
    driver.get(Stage3.iloc[rt]['Offering Link'])

    # Wait for the parent element to be present
    #driver.implicitly_wait(1)

    # Find all the parent elements
    parent_elements = driver.find_elements(By.CSS_SELECTOR, '.SchoolServicesAndOfferingsGroupContent')

    # Iterate over each parent element
    for parent_element in parent_elements:
        try:
            # Find the description within the parent element
            description = parent_element.find_element(By.CSS_SELECTOR, '.SchoolServicesAndOfferingsDescription').get_attribute("innerText")

            # Find all the values within the parent element
            values = parent_element.find_elements(By.CSS_SELECTOR, 'ul.SchoolServicesAndOfferingsValues li')

            # Extract the texts and their corresponding status
            for value in values:
                try: 
                    text = value.get_attribute("innerText")
                    descservicesoffering = description + "-" + text

                    if descservicesoffering in masterservicesofferinglist:
                        if 'Disabled' in value.get_attribute('class'):
                            pass
                        else:
                            Stage3.loc[rt,descservicesoffering] = 1
                    else:
                        masterservicesofferinglist.append(descservicesoffering)
                        Stage3[descservicesoffering] = 0

                        if 'Disabled' in value.get_attribute('class'):
                            pass
                        else:
                            Stage3.loc[rt,descservicesoffering] = 1       

                except StaleElementReferenceException:

                    # Handle the exception if the element becomes stale
                    print("StaleElementReferenceException occurred. Future Retry Needed.")
                    Stage3.loc[rt,'retry3'] = 1
        except StaleElementReferenceException:

        # Handle the exception if the element becomes stale
            print("StaleElementReferenceException occurred. Future Retry Needed.")
            Stage3.loc[rt,'retry3'] = 1
            
    # Close the WebDriver
    driver.quit()  

Reprocessing District ID: 75050 School Name: Riverside Elementary School
Reprocessing District ID: 75070 School Name: Central Elementary School
Reprocessing District ID: 76070 School Name: Carsonville-Port Sanilac Learning Center
Reprocessing District ID: 78110 School Name: Bentley Bright Beginnings
Reprocessing District ID: 79020 School Name: Caro Middle School
Reprocessing District ID: 79020 School Name: Caro Alternative High School
Reprocessing District ID: 79030 School Name: Transition Center
Reprocessing District ID: 79090 School Name: Mayville Elementary School
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry Needed.
StaleElementReferenceException occurred. Future Retry 

Reprocessing District ID: 82929 School Name: East Arbor Charter Academy
Reprocessing District ID: 82930 School Name: East Arbor Charter Academy
Reprocessing District ID: 82947 School Name: East Arbor Charter Academy
Reprocessing District ID: 82969 School Name: East Arbor Charter Academy
Reprocessing District ID: 82974 School Name: East Arbor Charter Academy
Reprocessing District ID: 82981 School Name: East Arbor Charter Academy
Reprocessing District ID: 82985 School Name: East Arbor Charter Academy
Reprocessing District ID: 83000 School Name: East Arbor Charter Academy
Reprocessing District ID: 83000 School Name: East Arbor Charter Academy
Reprocessing District ID: 83010 School Name: East Arbor Charter Academy
Reprocessing District ID: 84010 School Name: East Arbor Charter Academy


In [26]:
retry_rows3 = Stage3[Stage3['retry3'] == 1].index.tolist()

In [28]:
for rt in retry_rows3:
    
    # Perform some operation on each school
    print("Reprocessing District ID:", Stage3.iloc[rt]['District ID'], "School Name:", Stage3.iloc[rt]['School Name'])
    
    # Set up the Selenium driver
    service = Service('C:\Program Files\chromedriver.exe')  # Specify the path to your chromedriver executable
    options = Options()
    options.add_argument('--headless')  # Run the browser in headless mode (without GUI)
    driver = webdriver.Chrome(service=service, options=options)
    
    #Identifying the appropriate website
    driver.get(Stage3.iloc[rt]['Offering Link'])

    # Wait for the parent element to be present
    #driver.implicitly_wait(1)

    # Find all the parent elements
    parent_elements = driver.find_elements(By.CSS_SELECTOR, '.SchoolServicesAndOfferingsGroupContent')

    # Iterate over each parent element
    for parent_element in parent_elements:
        try:
            # Find the description within the parent element
            description = parent_element.find_element(By.CSS_SELECTOR, '.SchoolServicesAndOfferingsDescription').get_attribute("innerText")

            # Find all the values within the parent element
            values = parent_element.find_elements(By.CSS_SELECTOR, 'ul.SchoolServicesAndOfferingsValues li')

            # Extract the texts and their corresponding status
            for value in values:
                try: 
                    text = value.get_attribute("innerText")
                    descservicesoffering = description + "-" + text

                    if descservicesoffering in masterservicesofferinglist:
                        if 'Disabled' in value.get_attribute('class'):
                            pass
                        else:
                            Stage3.loc[rt,descservicesoffering] = 1
                    else:
                        masterservicesofferinglist.append(descservicesoffering)
                        Stage3[descservicesoffering] = 0

                        if 'Disabled' in value.get_attribute('class'):
                            pass
                        else:
                            Stage3.loc[rt,descservicesoffering] = 1       

                except StaleElementReferenceException:

                    # Handle the exception if the element becomes stale
                    print("StaleElementReferenceException occurred. Future Retry Needed.")
                    Stage3.loc[rt,'retry3'] = 1
        except StaleElementReferenceException:

        # Handle the exception if the element becomes stale
            print("StaleElementReferenceException occurred. Future Retry Needed.")
            Stage3.loc[rt,'retry4'] = 1
            
    # Close the WebDriver
    driver.quit()  

Reprocessing District ID: 79090 School Name: Mayville Elementary School
Reprocessing District ID: 80110 School Name: Gobles High School
Reprocessing District ID: 81902 School Name: Central Academy
Reprocessing District ID: 82095 School Name: East Arbor Charter Academy
Reprocessing District ID: 82110 School Name: East Arbor Charter Academy
Reprocessing District ID: 82365 School Name: East Arbor Charter Academy
Reprocessing District ID: 82717 School Name: East Arbor Charter Academy


In [30]:
Stage3.to_csv(f"{data_home}/SchoolServicesOffering_10_30_23.csv", index=True)